In [2]:
import pandas as pd

## Read data

FMA repository is [here](https://github.com/mdeff/fma)

- Download the metadata file [here](https://os.unil.cloud.switch.ch/fma/fma_metadata.zip) and unzip to `data-FMA` folder
- `tracks.csv` contains album, name, genre, and tags
- only a portion have complete tag information

## Read FMA tracks subset

In [3]:
filePath = "data-processed/tracks-tags.csv"
df = pd.read_csv(filePath, header=[0,1], index_col=0)
df.head()

artist            album                          track           \
                name            title                          title   genres   
track_id                                                                        
137           Airway     Live at LACE                         Side A  [1, 32]   
138           Airway     Live at LACE                         Side B  [1, 32]   
850       Human Host  Exploding Demon                Tomb Of Science     [12]   
851       Human Host  Exploding Demon                     Six Realms     [12]   
852       Human Host  Exploding Demon  Escape From the Organ Chamber     [12]   

                                      
           genres_all           tags  
track_id                              
137       [32, 1, 38]      ['lafms']  
138       [32, 1, 38]      ['lafms']  
850              [12]  ['baltimore']  
851              [12]  ['baltimore']  
852              [12]  ['baltimore']

## Note: this is a multiindex dataframe

Some of examples of how to select columns:

In [4]:
df.columns # See what the column names are

MultiIndex([('artist',       'name'),
            ( 'album',      'title'),
            ( 'track',      'title'),
            ( 'track',     'genres'),
            ( 'track', 'genres_all'),
            ( 'track',       'tags')],
           )

The column names are [tuples](https://docs.python.org/3/tutorial/datastructures.html#tuples-and-sequences) (like lists that cannot be modified).

To access a column:

In [5]:
df[("album", "title")]

track_id
137                    Live at LACE
138                    Live at LACE
850                 Exploding Demon
851                 Exploding Demon
852                 Exploding Demon
                    ...            
155269                     Volatile
155275                     Dog Wave
155276                     Dog Wave
155277                     Dog Wave
155320    What I Tell Myself Vol. 2
Name: (album, title), Length: 23496, dtype: object

To access multiple columns:

In [6]:
df[[("album", "title"), ("track", "tags")]]

,album,track
,title,tags
track_id,,
137,Live at LACE,['lafms']
138,Live at LACE,['lafms']
850,Exploding Demon,['baltimore']
851,Exploding Demon,['baltimore']
852,Exploding Demon,['baltimore']
...,...,...
155269,Volatile,"['dark ambient', 'dark', 'ambient', 'noise', '..."
155275,Dog Wave,"['noise', 'stretching is magic', 'free music',..."


## TODO: replace or map genre codes with names

ref: pandas [replace](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html#pandas.DataFrame.replace) or [map](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.map.html#pandas.DataFrame.map)

In [8]:
# Read in genres.csv

df_genres = pd.read_csv("data-FMA/genres.csv")
df_genres.head()

,genre_id,#tracks,parent,title,top_level
0,1,8693,38,Avant-Garde,38
1,2,5271,0,International,2
2,3,1752,0,Blues,3
3,4,4126,0,Jazz,4
4,5,4106,0,Classical,5


In [23]:
dict_map = {}

for index, row in df_genres.iterrows():
    dict_map[row["genre_id"]] = row["title"]

dict_map

{1: 'Avant-Garde',
 2: 'International',
 3: 'Blues',
 4: 'Jazz',
 5: 'Classical',
 6: 'Novelty',
 7: 'Comedy',
 8: 'Old-Time / Historic',
 9: 'Country',
 10: 'Pop',
 11: 'Disco',
 12: 'Rock',
 13: 'Easy Listening',
 14: 'Soul-RnB',
 15: 'Electronic',
 16: 'Sound Effects',
 17: 'Folk',
 18: 'Soundtrack',
 19: 'Funk',
 20: 'Spoken',
 21: 'Hip-Hop',
 22: 'Audio Collage',
 25: 'Punk',
 26: 'Post-Rock',
 27: 'Lo-Fi',
 30: 'Field Recordings',
 31: 'Metal',
 32: 'Noise',
 33: 'Psych-Folk',
 36: 'Krautrock',
 37: 'Jazz: Vocal',
 38: 'Experimental',
 41: 'Electroacoustic',
 42: 'Ambient Electronic',
 43: 'Radio Art',
 45: 'Loud-Rock',
 46: 'Latin America',
 47: 'Drone',
 49: 'Free-Folk',
 53: 'Noise-Rock',
 58: 'Psych-Rock',
 63: 'Bluegrass',
 64: 'Electro-Punk',
 65: 'Radio',
 66: 'Indie-Rock',
 70: 'Industrial',
 71: 'No Wave',
 74: 'Free-Jazz',
 76: 'Experimental Pop',
 77: 'French',
 79: 'Reggae - Dub',
 81: 'Afrobeat',
 83: 'Nerdcore',
 85: 'Garage',
 86: 'Indian',
 88: 'New Wave',
 89: 'P

['[1,15]']

## TODO: parse the cells in the track-tags column

ref: python [strings](https://docs.python.org/3/library/string.html#module-string) and the [re](https://docs.python.org/3/library/re.html#module-re) package

- practice re [here](https://regex101.com)

In [ ]:
def myFunction(aString):

    return aString.split()

x = "[1, 15, 40]"

myFunction(x)

In [27]:
def replace_numbers(input_string, replacement_dict):
    result = ''
    for char in input_string:
        if char.isdigit():
            result += replacement_dict.get(int(char), char)
        else:
            result += char
    return result

# Example usage:
replacement_dict = {1: 'One', 2: 'Two', 3: 'Three', 4: 'Four', 5: 'Five'}
input_string = "I have 123 apples and 45 oranges."

output_string = replace_numbers(input_string, replacement_dict)
print(output_string)

I have OneTwoThree apples and FourFive oranges.
